In [1]:
import numpy as np
import pandas as pd
import scipy.io
import scanpy.api as sc

import gensim
from gensim import corpora, models, similarities
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

## Reading in Data

Using scipy to read in the sparse matrix formated data.

In [2]:
project_path = '/ahg/regevdata/projects/FASI_DOmice/'
my_path = project_path + 'kirk/'
sc_path = project_path + 'scRNA_seq/cellranger/counts/C1/outs/mm10'

In [ ]:
## Reading 
geneids = pd.read_csv(sc_path + 'genes.tsv', delimiter = '\t', names = ['id','symbol'])
mat = scipy.io.mmread(sc_path + 'matrix.mtx')
mat = mat.tocsr()

In [3]:
## or using scanpy's AnnData
adata = sc.read(my_path + 'data/scanpy/allchannels.h5ad')
geneids = adata.var_names
mat = adata.X.transpose(copy=True)

## Filtering

There are a lot of empty cells(documents) and cells with very few genes (words) expressed.  Filtering out to require at least 200 genes per cell.

In [ ]:
## keeping cells that have at least 200 Genes expressed
keep_ind = mat.getnnz(axis=0) > 200
mat = mat[:, np.where(keep_ind)[0]]

In [ ]:
## Using only highly variable genes
highly_variable = sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
mat = mat[highly_variable.highly_variable,:]

## Converting Data

Converting sparse matrix to a corpus to use with gensim

In [ ]:
corpus = gensim.matutils.Sparse2Corpus(mat)
## saving as a corpus file into tmp directory
corpora.MmCorpus.serialize('/tmp/corpus.mm', corpus)

# loading corpus
# corpus = corpora.MmCorpus('/tmp/corpus.mm')

In [ ]:
id_list = geneids['id'].tolist()
out = [[]]
for i in id_list: out.append([i])

dictionary = corpora.Dictionary(out)
dictionary.save('/tmp/genes_dictionary.dict')  # store the dictionary, for future reference

## Read in saved dictionary
## dictionary = gensim.corpora.Dictionary.load(my_path + 'data/reference/mouse_gene_dictionary.dict')

## Running the Model

The following two chunks runs either LDA or HDP.  HDP chooses the number of topics for you. 

In [ ]:
# Latent Dirichlet Allocation
lda = models.LdaModel(corpus=corpus,
                      id2word=dictionary,
                      num_topics=t,
                      random_state=100,
                      # update_every=0, # only for ldamodel
                      # chunksize=10000,
                      # passes=1,
                      # alpha='auto',
                      # per_word_topics=True
                     )

## lda.print_topics(2) # print first 2 topics

In [ ]:
# Hierarchical Dirchlet Process
hdp = models.HdpModel(corpus, id2word=dictionary)
## hdp.print_topics(2) # print first 2 topics

## Exploring Topics

In [ ]:
for index, topic in lda.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(index, [w[0] for w in topic]))

In [ ]:
## Save top genes per topic to csv
top_words_per_topic = []
for t in range(lda.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda.show_topic(t, topn = 20)])

pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_genes.csv")

## Visualizing

Visualizing the topics with pyLDAvis

In [ ]:
vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

## Checking Number of Topics
Looping through a few different number of topics to see which are the most coherent.  The higher the better.

In [ ]:
topics_start = 4
topics_end = 30
step = 2
d = []
## loop through number of topics
for t in range(topics_start, topics_end, step):
    # Latent Dirichlet Allocation
    # Build LDA model
    print('running lda with '+ str(t) + ' topics \n')
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=t,
                                                random_state=100,
                                                # update_every=0, # only for ldamodel
                                                # chunksize=10000,
                                                # passes=1,
                                                # alpha='auto',
                                                # per_word_topics=True
                                               )



    ## Save Model
    print('saving lda model \n')
    lda_model.save(my_path + 'results/gensim/lda_allchannels_' + str(t) + '_topics')

    
    # Compute Perplexity
    print('calculating perplexity \n')
    perplex = lda_model.log_perplexity(corpus)  # a measure of how good the model is. lower the better.


    # Compute Coherence Score u_mass
    print('calculating coherence using u_mass \n')
    coherence_model = CoherenceModel(model=lda_model, corpus = corpus, dictionary = dictionary, coherence ='u_mass')
    coherence_values = coherence_model.get_coherence()

    d.append({'topics': t, 'perplexity': perplex, 'coherence': coherence_values})


## Save data frame of results
print('saving model statistic output')
df = pd.DataFrame(d)
df.to_csv(my_path + 'results/gensim/lda_model_build_results.csv')

In [ ]:
import matplotlib.pyplot as plt

limit=40; start=2; step=6;
x = range(topics_start, topics_end, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()